In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv('BH.csv')
df


In [ ]:
df.isna().sum()

In [ ]:
df.drop(columns=['area_type','availability','society','balcony'],inplace=True)

In [ ]:
df

In [ ]:
df['location']=df['location'].fillna('Sarjapur Road')

In [ ]:
df['size']=df['size'].fillna('2 BHK')

In [ ]:
df['bath']=df['bath'].fillna(df['bath'].median())

In [ ]:
df.info()

In [ ]:
df['bhk']=df['size'].str.split().str.get(0).astype(int)
df['bhk']

In [ ]:
def con(x):
    temp=x.split('-')
    if len(temp)==2:
        return (float(temp[0])+float(temp[1]))/2
    try: 
        return float(x)
    except:
        return None

In [ ]:
df['total_sqft']=df['total_sqft'].apply(con)

In [ ]:
df['total_sqft']

In [ ]:
df['price_s']=df['price']*100000/df['total_sqft']

In [ ]:
df['price_s']

In [ ]:
df['location']=df['location'].apply(lambda x: x.strip())
l_c=df['location'].value_counts()
l_c

In [ ]:
l_c10=l_c[l_c<=10]
l_c10

In [ ]:
df['location']=df['location'].apply(lambda x:'other' if x in l_c10 else x)

In [ ]:
df=df[((df['total_sqft']/df['bhk'])>=300)]

In [ ]:
df.describe()

In [ ]:
def r_o(df):
    d_o=pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.price_s)
        st=np.std(subdf.price_s)
        gen_df=subdf[(subdf.price_s>(m-st)) & (subdf.price_s<=(m+st))]
        d_o=pd.concat([d_o,gen_df],ignore_index=True)
    return d_o
data=r_o(df)
data.describe()

In [ ]:
def b_o(df):
    e_i=np.array([])
    for l,l_df in df.groupby('location'):
        b_s={}
        for bhk,bhk_df in l_df.groupby('bhk'):
            b_s[bhk]={
                'mean':np.mean(bhk_df.price_s),
                'std':np.std(bhk_df.price_s),
                'count':bhk_df.shape[0]
            }
        for bhk,bhk_df in l_df.groupby('bhk'):
            stats=b_s.get(bhk-1)
            if(stats and stats['count']>5):
                e_i=np.append(e_i,bhk_df[bhk_df.price_s<(stats['mean'])].index.values)
    return df.drop(e_i,axis='index')

In [ ]:
data=b_o(data)


In [ ]:
data

In [ ]:
data.drop(columns=['size','price_s'],inplace=True)

In [ ]:
x=data.drop(columns=['price'])
y=data['price']
y

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [ ]:
x_t,x_te,y_t,y_te=train_test_split(x,y,test_size=0.2,random_state=0)
data

In [ ]:
c_t=make_column_transformer((OneHotEncoder(sparse_output=False),['location']),remainder='passthrough')

In [ ]:
scaler=StandardScaler()
lr=LinearRegression()

In [ ]:
pipe=make_pipeline(c_t,scaler,lr)

In [ ]:
pipe.fit(x_t,y_t)

In [ ]:
y_p_lr=pipe.predict(x_te)

In [ ]:
r2_score(y_te,y_p_lr)

In [ ]:
la=Lasso()
pipe=make_pipeline(c_t,scaler,la)
pipe.fit(x_t,y_t)


In [ ]:
y_p_la=pipe.predict(x_te)

In [ ]:
r2_score(y_te,y_p_la)

In [ ]:
ri=Ridge()
pipe=make_pipeline(c_t,scaler,ri)
pipe.fit(x_t,y_t)


In [ ]:

y_p_ri=pipe.predict(x_te)


In [ ]:
r2_score(y_te,y_p_ri)

In [ ]:
import pickle
pickle.dump(pipe,open('ri.pkl','wb'))

In [ ]:
data
data.describe()
data

In [ ]:
data.to_csv('c.csv')